In [114]:
import numpy as np
from scipy.spatial import SphericalVoronoi,distance_matrix
from scipy.optimize import basinhopping,minimize

from matplotlib import colors
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

from mayavi import mlab
mlab.init_notebook()

from tqdm import tqdm

%matplotlib notebook

Notebook initialized with ipy backend.


In [12]:
def sample_spherical(npoints, ndim=3):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    return vec.transpose()
def arc(p1,p2,t):
    a0=p1[0]+t*(p2[0]-p1[0])
    a1=p1[1]+t*(p2[1]-p1[1])
    a2=p1[2]+t*(p2[2]-p1[2])
    n=(a0**2+a1**2+a2**2)**.5
    return np.array([a0/n,a1/n,a2/n]).transpose()

In [13]:
np.shape(arc(np.array([0,0,1]),np.array([1,0,0]),np.linspace(0,1,10)))

(10, 3)

In [133]:
# set input data
N=20
points = sample_spherical(N)
s=np.shape(points)

def normalize(points):
    norm=np.sum(points**2,axis=1)**.5
    points[:,0]/=norm
    points[:,1]/=norm
    points[:,2]/=norm
    return points
    


def E(pointsFlat):
    points=normalize(np.reshape(pointsFlat,s))
    M=distance_matrix(points,points)
    inv=1/(M+1e-6)
    index=np.arange(len(M))
    inv[index,index]=0
    return np.sum(inv)
    

points=normalize(np.reshape(minimize(E,points.flatten()).x,s))

In [110]:
points

array([[ 0.9000618 , -0.41854976,  0.12126355],
       [ 0.51402068,  0.47074764, -0.71706304],
       [ 0.27200074, -0.59492997, -0.75635569],
       [ 0.08899584,  0.05268522,  0.99463763],
       [-0.29704506,  0.94198275,  0.15630971],
       [-0.02853526, -0.89054465,  0.45399997],
       [-0.71331304, -0.61917261, -0.32834399],
       [-0.57448076,  0.36711149, -0.73157433],
       [ 0.71331262,  0.61917234,  0.32834543],
       [-0.8750181 ,  0.07149724,  0.47878123]])

In [134]:
radius = 1
center = np.array([0, 0, 0])
sv = SphericalVoronoi(points, radius, center)
sv.sort_vertices_of_regions()

In [135]:
# plot the unit sphere for reference (optional)
mlab.figure(size=(700,700))


for region in tqdm(sv.regions):
    t=np.linspace(0,1,100)
    for i in range(len(region)):
        a=arc(sv.vertices[region[i]],sv.vertices[region[(i+1)%len(region)]],t)
        mlab.plot3d(a[:,0]*1.01,a[:,1]*1.01,a[:,2]*1.01,color=(0.9,0.7,0.7),tube_radius=0.01)
mlab.points3d(points[:, 0], points[:, 1], points[:, 2],scale_factor=.05,color=(0.2,0.2,1))
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
mlab.mesh(x, y, z)


100%|██████████| 20/20 [00:02<00:00,  7.70it/s]


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xbc\x00\x00\x02\xbc\x08\x02\x00\x00\x00\x82\xd8\…